In [1]:
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from scipy import spatial
%matplotlib inline

# Project 3: Word2Vec (70 pt)
The goal of this project is to obtain the vector representations for words from text.

The main idea is that words appearing in similar contexts have similar meanings. Because of that, word vectors of similar words should be close together. Models that use word vectors can utilize these properties, e.g., in sentiment analysis a model will learn that "good" and "great" are positive words, but will also generalize to other words that it has not seen (e.g. "amazing") because they should be close together in the vector space.

Vectors can keep other language properties as well, like analogies. The question "a is to b as c is to ...?", where the answer is d, can be answered by looking into word vector space and calculating $\mathbf{u}_b - \mathbf{u}_a + \mathbf{u}_c$, and finding the word vector that is the closest to the result.

## Your task
Complete the missing code in this notebook. Make sure that all the functions follow the provided specification, i.e. the output of the function exactly matches the description in the docstring. 

We are given a text that contains $N$ unique words $\{ x_1, ..., x_N \}$. We will focus on the Skip-Gram model in which the goal is to predict the context window $S = \{ x_{i-l}, ..., x_{i-1}, x_{i+1}, ..., x_{i+l} \}$ from current word $x_i$, where $l$ is the window size. 

We get a word embedding $\mathbf{u}_i$ by multiplying the matrix $\mathbf{U}$ with a one-hot representation $\mathbf{x}_i$ of a word $x_i$. Then, to get output probabilities for context window, we multiply this embedding with another matrix $\mathbf{V}$ and apply softmax. The objective is to minimize the loss: $-\mathop{\mathbb{E}}[P(S|x_i;\mathbf{U}, \mathbf{V})]$.

You are given a dataset with positive and negative reviews. Your task is to:
+ Construct input-output pairs corresponding to the current word and a word in the context window
+ Implement forward and backward propagation with parameter updates for Skip-Gram model
+ Train the model
+ Test it on word analogies and sentiment analysis task

## General remarks

Fill in the missing code at the markers
```python
###########################
# YOUR CODE HERE
###########################
```
Do not add or modify any code at other places in the notebook except where otherwise explicitly stated.
After you fill in all the missing code, restart the kernel and re-run all the cells in the notebook.

The following things are **NOT** allowed:
- Using additional `import` statements
- Copying / reusing code from other sources (e.g. code by other students)

If you plagiarise even for a single project task, you won't be eligible for the bonus this semester.

# 1. Load data (5 pts)

We'll be working with a subset of reviews for restaurants in Las Vegas. The reviews that we'll be working with are either 1-star or 5-star.

In [2]:
data = np.load("task03_data.npy", allow_pickle=True).item()
reviews_1star = [[x.lower() for x in s] for s in data["reviews_1star"]]
reviews_5star = [[x.lower() for x in s] for s in data["reviews_5star"]]

We generate the vocabulary by taking the top 200 words by their frequency from both positive and negative sentences. We could also use the whole vocabulary, but that would be slower. Other words are represented with "unk".

In [3]:
corpus = reviews_1star + reviews_5star
vocabulary = [w for s in corpus for w in s]
vocabulary, counts = zip(*Counter(vocabulary).most_common(200))

In [4]:
corpus = [[w if w in vocabulary else 'unk' for w in s] for s in corpus]
vocabulary += ('unk',) # Add "unk" to vocabulary
counts += (sum([w == 'unk' for s in corpus for w in s]),) # Add count for "unk"

In [5]:
VOCABULARY_SIZE = len(vocabulary)
EMBEDDING_DIM = 32

In [6]:
print('Number of positive reviews:', len(reviews_1star))
print('Number of negative reviews:', len(reviews_5star))
print('Number of unique words:', VOCABULARY_SIZE)

Number of positive reviews: 1000
Number of negative reviews: 2000
Number of unique words: 201


You have to create two dictionaries: `word_to_ind` and `ind_to_word` so we can go from text to numerical representation and vice versa. The input into the model will be the index of the word denoting the position in the vocabulary.

In [7]:
"""
Implement
---------
word_to_ind: dict
    The keys are words (str) and the value is the corresponding position in the vocabulary
ind_to_word: dict
    The keys are indices (int) and the value is the corresponding word from the vocabulary
ind_to_freq: dict
    The keys are indices (int) and the value is the corresponding count in the vocabulary
"""

### BEGIN SOLUTION ###
word_to_ind = { w: i for i,w in enumerate(vocabulary) }
ind_to_word = { i: w for i,w in enumerate(vocabulary) }
ind_to_freq = { i: counts[i] for i,w in enumerate(vocabulary) }
### END SOLUTION ###

In [8]:
print(f'Word \"the\" is at position {word_to_ind["the"]} appearing {ind_to_freq[word_to_ind["the"]]} times')
# Should output: 
# Word "the" is at position 0 appearing 2017 times

Word "the" is at position 0 appearing 2017 times


# 2. Create word pairs (5 pts)

We need all the word pairs $\{ x_i, x_j \}$, where $x_i$ is the current word and $x_j$ is from its context window. These will correspond to input-output pairs. We want them to be represented numericaly so you should use `word_to_ind` dictionary.

In [9]:
def get_window(sentence, window_size):
    pairs = []

    """
    Iterate over all the sentences
    Take all the words from [i - window_size) to (i + window_size] and save them to pairs
    
    Parameters
    ----------
    sentence: list
        A list of sentences, each sentence containing a list of words of str type
    window_size: int
        A positive scalar
        
    Returns
    -------
    pairs: list
        A list of tuple (word index, word index from its context) of int type
    """

    ### BEGIN SOLUTION ###
    for i in range(len(sentence)):
        for j in range(max(0, i - window_size), min(len(sentence), i + window_size + 1)):
            if i != j:
                xi, xj = sentence[i], sentence[j]
                pairs.append((word_to_ind[xi], word_to_ind[xj]))
    ### END SOLUTION ###

    return pairs

In [10]:
data = []
for x in corpus:
    data += get_window(x, window_size=3)
data = np.array(data)

print('First 5 pairs:', data[:5].tolist())
print('Total number of pairs:', data.shape[0])
# Should output:
# First 5 pairs: [[10, 200], [10, 6], [10, 64], [200, 10], [200, 6]]
# Total number of pairs: 207462

First 5 pairs: [[10, 200], [10, 6], [10, 64], [200, 10], [200, 6]]
Total number of pairs: 207462


We calculate a weighting score to counter the imbalance between the rare and frequent words. Rare words will be sampled more frequently. See https://arxiv.org/pdf/1310.4546.pdf

In [11]:
probabilities = [1 - np.sqrt(1e-3 / ind_to_freq[x]) for x in data[:,0]]
probabilities /= np.sum(probabilities)
print(probabilities[:3])
# Should output: 
# [4.8206203e-06 4.8206203e-06 4.8206203e-06]

[4.8206203e-06 4.8206203e-06 4.8206203e-06]


# 3. Model definition (55 pts)

In this part you should implement forward and backward propagation together with update of the parameters i.e.:
+ One-hot encoding of the words(5 pts)
+ Softmax (5 pts)
+ Loss implementation & computation (5 pts)
+ Forward pass (15 pts)
+ Backward pass (15 pts)
+ Optimizer (10 pts)

In [12]:
class Embedding():
    """
    Word embedding model.

    Parameters
    ----------
    N: int
        Number of unique words in the vocabulary
    D: int
        Dimension of the word vector embedding
    """
    def __init__(self, N, D):
        self.N = N
        self.D = D

        self.ctx = None # Used to store values for backpropagation

        self.U = None
        self.V = None
        self.reset_parameters()

    def reset_parameters(self):
        """
        We initialize weight matrices U and V of dimension (D, N) and (N, D), respectively
        """
        self.ctx = None
        self.U = np.random.normal(0, np.sqrt(6. / (self.D + self.N)), (self.D, self.N))
        self.V = np.random.normal(0, np.sqrt(6. / (self.D + self.N)), (self.N, self.D))

    def one_hot(self, x, N):
        """
        Given a vector returns a matrix with rows corresponding to one-hot encoding.
        
        Parameters
        ----------
        x: array
            M-dimensional vector containing integers from [0, N]
        N: int
            Number of posible classes
        
        Returns
        -------
        one_hot: array
            (N, M) matrix where each column is N-dimensional one-hot encoding of elements from x 
        """

        ### BEGIN SOLUTION ###
        one_hot = np.zeros((N, x.shape[0]))
        one_hot[x, np.arange(x.shape[0])] = 1
        ### END SOLUTION ###

        assert one_hot.shape == (N, x.shape[0]), 'Incorrect one-hot embedding shape'
        return one_hot

    def softmax(self, x, axis):
        """
        Parameters
        ----------
        x: array
            A non-empty matrix of any dimension
        axis: int
            Dimension on which softmax is performed
            
        Returns
        -------
        y: array
            Matrix of same dimension as x with softmax applied to 'axis' dimension
        """
        
        # Note! You should implement a numerically stable version of softmax
        
        ### BEGIN SOLUTION ###
        x_stable = x - np.max(x, axis=axis, keepdims=True)
        y = np.exp(x_stable) / np.sum(np.exp(x_stable), axis=axis, keepdims=True)
        ### END SOLUTION ###
        
        assert x.shape == y.shape, 'Output should have the same shape is input'
        return y

    def loss(self, y, prob):
        """
        Parameters
        ----------
        y: array
            (N, M) matrix of M samples where columns are one-hot vectors for true values
        prob: array
            (N, M) column of M samples where columns are probability vectors after softmax

        Returns
        -------
        loss: int
            Cross-entropy loss calculated as: 1 / M * sum_i(sum_j(y_ij * log(prob_ij)))
        """

        prob = np.clip(prob, 1e-8, None)
        
        ### BEGIN SOLUTION ###
        loss = -np.sum(y * np.log(prob)) / y.shape[1]
        ### END SOLUTION ###
        
        assert isinstance(loss, float), 'Loss must be a scalar'
        return loss

    def forward(self, x, y):
        """
        Performs forward and backward propagation and updates weights
        
        Parameters
        ----------
        x: array
            M-dimensional mini-batched vector containing input word indices of int type
        y: array
            Output words, same dimension and type as 'x'
        learning_rate: float
            A positive scalar determining the update rate
            
        Returns
        -------
        loss: float
            Cross-entropy loss
        d_U: array
            Partial derivative of loss w.r.t. U
        d_V: array
            Partial derivative of loss w.r.t. V
        """
        
        # Input transformation
        """
        Input is represented with M-dimensional vectors
        We convert them to (N, M) matrices such that columns are one-hot 
        representations of the input
        """
        x = self.one_hot(x, self.N)
        y = self.one_hot(y, self.N)
        
        # Forward propagation
        """
        Returns
        -------
        embedding: array
            (D, M) matrix where columns are word embedding from U matrix
        logits: array
            (N, M) matrix where columns are output logits
        prob: array
            (N, M) matrix where columns are output probabilities
        """
        
        ### BEGIN SOLUTION ###
        embedding = np.matmul(self.U, x)
        logits = np.matmul(self.V, embedding)
        prob = self.softmax(logits, axis=0)
        ### END SOLUTION ###

        assert embedding.shape == (self.D, x.shape[1])
        assert logits.shape == (self.N, x.shape[1])
        assert prob.shape == (self.N, x.shape[1])
        
        # Save values for backpropagation
        self.ctx = (embedding, logits, prob, x, y)
        
        # Loss calculation
        loss = self.loss(y, prob)
        
        return loss
        
    def backward(self):
        """
        Given parameters from forward propagation, returns gradient of U and V.
        
        Returns
        -------
        d_V: array
            (D, N) matrix of partial derivatives of loss w.r.t. V
        d_U: array
            (N, D) matrix of partial derivatives of loss w.r.t. U
        """

        embedding, logits, prob, x, y = self.ctx

        ### BEGIN SOLUTION ###
        d_logits = prob - y
        d_embedding = np.matmul(self.V.T, d_logits)
        M = y.shape[1]
        d_V = np.matmul(d_logits, embedding.T) / M
        d_U = np.matmul(d_embedding, x.T) / M
        ### END SOLUTION ###

        assert d_V.shape == (self.N, self.D)
        assert d_U.shape == (self.D, self.N)

        return { 'V': d_V, 'U': d_U }

In [13]:
class Optimizer():
    """
    Stochastic gradient descent with momentum optimizer.

    Parameters
    ----------
    model: object
        Model as defined above
    learning_rate: float
        Learning rate
    momentum: float (optional)
        Momentum factor (default: 0)
    """
    def __init__(self, model, learning_rate, momentum=0):
        self.model = model
        self.learning_rate = learning_rate
        self.momentum = momentum
        
        self.previous = None # Previous gradients
    
    def _init_previous(self, grad):
        # Initialize previous gradients to zero
        self.previous = { k: np.zeros_like(v) for k,v in grad.items() }
    
    def step(self, grad):
        if self.previous is None:
            self._init_previous(grad)
            
        for name, dw in grad.items():
            dw_prev = self.previous[name]
            w = getattr(self.model, name)

            """
            Given weight w, previous gradients dw_prev and current 
            gradients dw, performs an update of weight w.

            Returns
            -------
            dw_new: array
                New gradients calculated as combination of previous and
                current, weighted with momentum factor.
            w_new: array
                New weights calculated with a single step of gradient
                descent using dw_new.
            """
            ### BEGIN SOLUTION ###
            dw_new = self.momentum * dw_prev + (1 - self.momentum) * dw
            w_new = w - self.learning_rate * dw_new
            ### END SOLUTION ###

            self.previous[name] = dw_new
            setattr(self.model, name, w_new)

## 3.1 Gradient check

The following code checks whether the updates for weights are implemented correctly. It should run without an error.

In [14]:
def get_loss(model, old, variable, epsilon, x, y, i, j):
    np.random.seed(123)
    model.reset_parameters() # reset weights
    
    delta = np.zeros_like(old)
    delta[i, j] = epsilon
    
    setattr(model, variable, old + delta) # change one weight by a small amount
    loss = model.forward(x, y)

    return loss

def gradient_check_for_weight(model, variable, i, j, k, l):
    x, y = np.array([i, i]), np.array([j, j]) # set input and output
    
    np.random.seed(123)
    model.reset_parameters() # reset weights

    old = getattr(model, variable)
    
    loss = model.forward(x, y)
    grad = model.backward()
    
    eps = 1e-4
    loss_positive = get_loss(model, old, variable, eps, x, y, k, l) # loss for positive change on one weight
    loss_negative = get_loss(model, old, variable, -eps, x, y, k, l) # loss for negative change on one weight
    
    true_gradient = (loss_positive - loss_negative) / 2 / eps # calculate true derivative wrt one weight

    assert abs(true_gradient - grad[variable][k, l]) < 1e-5, 'Incorrect gradient'

def gradient_check():
    N, D = VOCABULARY_SIZE, EMBEDDING_DIM
    model = Embedding(N, D)

    # check for V
    for _ in range(20):
        i, j, k = [np.random.randint(0, d) for d in [N, N, D]] # get random indices for input and weights
        gradient_check_for_weight(model, 'V', i, j, i, k)

    # check for U
    for _ in range(20):
        i, j, k = [np.random.randint(0, d) for d in [N, N, D]]
        gradient_check_for_weight(model, 'U', i, j, k, i)

    print('Gradients checked - all good!')

gradient_check()

Gradients checked - all good!


# 4. Training

We train our model using stochastic gradient descent. At every step we sample a mini-batch from data and update the weights.

The following function samples words from data and creates mini-batches. It subsamples frequent words based on previously calculated probabilities.

In [15]:
rng = np.random.default_rng(123)
def get_batch(data, size, prob):
    x = rng.choice(data, size, p=prob)
    return x[:,0], x[:,1]

Training the model can take some time so plan accordingly.

In [16]:
model = Embedding(N=VOCABULARY_SIZE, D=EMBEDDING_DIM)
optim = Optimizer(model, learning_rate=1.0, momentum=0.5)

losses = []

MAX_ITERATIONS = 15000
PRINT_EVERY = 1000
BATCH_SIZE = 1000

for i in range(MAX_ITERATIONS):
    x, y = get_batch(data, BATCH_SIZE, probabilities)
    
    loss = model.forward(x, y)
    grad = model.backward()
    optim.step(grad)
    
    assert not np.isnan(loss)
    
    losses.append(loss)

    if (i + 1) % PRINT_EVERY == 0:
        print(f'Iteration: {i + 1}, Avg. training loss: {np.mean(losses[-PRINT_EVERY:]):.4f}')

Iteration: 1000, Avg. training loss: 3.7223
Iteration: 2000, Avg. training loss: 3.5687
Iteration: 3000, Avg. training loss: 3.5505
Iteration: 4000, Avg. training loss: 3.5365
Iteration: 5000, Avg. training loss: 3.5251
Iteration: 6000, Avg. training loss: 3.5167
Iteration: 7000, Avg. training loss: 3.5131
Iteration: 8000, Avg. training loss: 3.5007
Iteration: 9000, Avg. training loss: 3.4970
Iteration: 10000, Avg. training loss: 3.4876
Iteration: 11000, Avg. training loss: 3.4943
Iteration: 12000, Avg. training loss: 3.4864
Iteration: 13000, Avg. training loss: 3.4802
Iteration: 14000, Avg. training loss: 3.4826
Iteration: 15000, Avg. training loss: 3.4813


The embedding matrix is given by $\mathbf{U}^T$, where the $i$th row is the vector for $i$th word in the vocabulary.

In [17]:
emb_matrix = model.U.T

# 5. Analogies (5 pts)

As mentioned before, vectors can keep some language properties like analogies. Given a relation a:b and a query c, we can find d such that c:d follows the same relation. We hope to find d by using vector operations. In this case, finding the real word vector $\mathbf{u}_d$ closest to $\mathbf{u}_b - \mathbf{u}_a + \mathbf{u}_c$ gives us d. 

**Note that the quality of the analy results is not expected to be excellent.**

In [18]:
triplets = [['is', 'was', 'were'], ['lunch', 'day', 'night'], ['i', 'my', 'your']]

for triplet in triplets:
    a, b, d = triplet

    """
    Returns
    
    Example: Paris (a) is to France (b) as _____ (c) is to Germany (d)
    
    -------
    result: array
        The embedding vector for word (c): w_a - w_b + w_d
    """

    ### BEGIN SOLUTION ###
    wa = emb_matrix[word_to_ind[a]]
    wb = emb_matrix[word_to_ind[b]]
    wd = emb_matrix[word_to_ind[d]]

    result = wa - wb + wd
    ### END SOLUTION ###

    distances = [spatial.distance.cosine(x, result) for x in emb_matrix]
    candidates = [ind_to_word[i] for i in np.argsort(distances)]
    candidates = [x for x in candidates if x not in [a, b, d]][:5]

    print(f'`{a}` is to `{b}` as [{", ".join(candidates)}] is to `{d}`')

`is` is to `was` as [are, sauce, rice, way, 3] is to `were`
`lunch` is to `day` as [dinner, great, experience, what, first] is to `night`
`i` is to `my` as [you, not, don't, what, if] is to `your`
